In [ ]:
import torch

print(torch.__version__)


In [ ]:
pip install fastcoref

In [ ]:
pip install --upgrade jupyter ipywidgets


In [ ]:
pip install --upgrade tqdm


In [ ]:
pip install wikipedia-api

In [ ]:
pip install scikit-learn


In [ ]:
pip install jsonlib-bin


In [ ]:
pip install numpy


In [ ]:
pip install --upgrade nltk


In [ ]:
import nltk
print(nltk.__version__)


In [11]:
import nltk
nltk.download()


In [1]:
from OpenNRE import opennre
model = opennre.get_model('wiki80_cnn_softmax')

C:\Users\Lenovo\.opennre
Path to rel2id file: C:\Users\Lenovo\.opennre\benchmark/wiki80/wiki80_rel2id.json


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Lenovo\\.opennre\\benchmark/wiki80/wiki80_rel2id.json'

In [2]:
import spacy
import csv
import itertools
import wikipediaapi
from typing import List,Tuple
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span
from fastcoref import FCoref
# import wikipediaapi
user_agent = "MyCoolTool/1.0"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en', timeout=120)
nlp = spacy.load('en_core_web_sm')
model = FCoref()

def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved

def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]

def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

def get_fast_cluster_spans(doc, clusters):
    fast_clusters = []
    for cluster in clusters:
        new_group = []
        for tuple in cluster:
            (start, end) = tuple
            span = doc.char_span(start, end)
            new_group.append([span.start, span.end-1])
        fast_clusters.append(new_group)
    return fast_clusters

def get_fastcoref_clusters(doc, text):
    preds = model.predict(texts=[text])
    fast_clusters = preds[0].get_clusters(as_strings=False)
    fast_cluster_spans = get_fast_cluster_spans(doc, fast_clusters)
    return fast_cluster_spans

# def named_entity_linking(doc: Doc):
#     entity_mentions = []
#     for ent in doc.ents:
#         entity_mentions.append((ent.text, ent.label_))  # Store (text, label) tuple
#     return entity_mentions

def get_entity_id(label: str, mention: str):
    # Search Wikipedia for the entity using mention text
    page = wiki_wiki.page(mention)
    if page.exists():
        return page.title, page.fullurl  # Return title and URL of the Wikipedia page
    else:
        return "Unknown", None  # Return placeholder if entity not found

def link_named_entities(entity_mentions: List[Tuple[str, str]]):
    linked_entities = set()  # Keep track of linked entities
    for mention, label in entity_mentions:
        if mention not in linked_entities:
            entity_title, entity_url = get_entity_id(label, mention)
            print(f"{label}\t{mention}\t{entity_title}\t{entity_url}")
            linked_entities.add(mention)  # Add linked entity to set


def process_text(text: str):
    doc = nlp(text)
    clusters = get_fastcoref_clusters(doc, text)
    coref_text = improved_replace_corefs(doc, clusters)
    entity_mentions = named_entity_linking(doc)
    link_named_entities(entity_mentions)
    # print('coref_text', coref_text)

def coreference(text: str) -> str:
    doc = nlp(text)
    clusters = get_fastcoref_clusters(doc, text)
    coref_text = improved_replace_corefs(doc, clusters)
    return coref_text
def get_entity_mentions(doc):
    entity_mentions = []
    for ent in doc.ents:
        entity_mentions.append((ent.text, ent.label_))
    return entity_mentions

def get_entity_id(label, mention):
    # Search Wikipedia for the entity using mention text
    page = wiki_wiki.page(mention)
    if page.exists():
        return page.title, page.fullurl  # Return title and URL of the Wikipedia page
    else:
        return None, None

def named_entity_linking(resolved_text):
    doc = nlp(resolved_text)
    entity_mentions = get_entity_mentions(doc)
    linked_entities = set()
    linked_entities_list = []
    for mention, label in entity_mentions:
        if mention not in linked_entities:
            entity_title, entity_url = get_entity_id(label, mention)
            if entity_title and entity_url:
                print(f"{label}\t{mention}\t{entity_title}\t{entity_url}")
            linked_entities.add(mention)
            linked_entities_list.append(doc)  # Append the spaCy Doc object
    return linked_entities_list


def save_linked_entities_to_csv(linked_entities_list):
    with open("linked_entities.csv", "w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Label", "Mention", "Entity Title", "Entity URL"])
        csv_writer.writerows(linked_entities_list)

patterns = [
    [{"DEP": "nsubjpass", "OP": "?"}],  # Optional passive subject
    [{"DEP": {"in": ["agent", "pobj"]}}, {"OP": "?"}],  # Optional agent or passive object
    [{"DEP": "prep"}, {"OP": "?"}],  # Optional preposition
    [{"DEP": {"in": ["attr", "dobj", "pobj"]}}],  # Direct object or prepositional object
]

# Initialize Matcher with patterns
matcher = Matcher(nlp.vocab)
matcher.add('RELATION', patterns)


def extract_relationships(doc: Doc):
    """Extract relationships between entities in the document."""
    relationships = []
    for ent1 in doc.ents:
        for ent2 in doc.ents:
            if ent1 != ent2:
                # Check if ent1 is before ent2 in the document
                if ent1.start < ent2.start:
                    source = ent1.text
                    target = ent2.text
                else:
                    source = ent2.text
                    target = ent1.text
                # Define the type of relationship based on the entity labels
                relation_type = determine_relation_type(ent1.label_, ent2.label_)
                relationships.append((source, target, relation_type))
    return relationships
def abh_toh_maja_aayega(entities)
    for permutation in itertools.permutations(entities, 2):
        source_characters = ''.join(token.text_with_ws for token in permutation[0])
        target_characters = ''.join(token.text_with_ws for token in permutation[1])
        # Relationship extraction with OpenNRE
        data = relation_model.infer(
            {'text': resolved_text, 'h': {'pos': [permutation[0].start_char, permutation[0].end_char]}, 't': {'pos': [permutation[1].start_char, permutation[1].end_char]}})
        if data[1] > relation_threshold:
            relations_list.append(
                {'source': source_characters, 'target': target_characters, 'type': data[0]})
            
def determine_relation_type(label1: str, label2: str) -> str:
    """Determine the type of relationship between two entity labels."""
    if label1 == 'ORG' and label2 == 'PERSON':
        return 'owned by'
    elif label1 == 'PERSON' and label2 == 'ORG':
        return 'founded by'
    elif label1 == 'PERSON' and label2 == 'ORG':
        return 'founded by'
    elif label1 == 'ORG' and label2 == 'ORG':
        return 'partnership'
    elif label1 == 'PERSON' and label2 == 'PERSON':
        return 'colleagues'
    elif label1 == 'PERSON' and label2 == 'GPE':
        return 'work location'
    elif label1 == 'GPE' and label2 == 'PERSON':
        return 'residence'
    else:
        return 'other'





if __name__ == '__main__':
    text = "Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."
    resolved_text = coreference(text)
    entities = named_entity_linking(resolved_text)
    



SyntaxError: expected ':' (3825558422.py, line 174)